In [6]:
import glob
import pydicom
import numpy as np
from PIL import Image

folder = r"C:\Users\haipw\Desktop\人工智能与医学影像\datasets"

# filenames = glob.glob(f"{folder}/train/*.dcm")
# print(filenames[3])
# dcm = pydicom.dcmread(filenames[3])
# # read pixel data
# arr = dcm.pixel_array
# print(arr.shape)
# print(dcm.PatientSex)
# # list all the dicom tags
# for i in dcm.elements():
#     if type(i) == pydicom.DataElement:
#         print(i.description)
# t = arr.flatten()

""" 
读取数据集
"""
def get_dataset(path:str):
    X = []
    y = []
    filenames = glob.glob(f"{path}/*.dcm")
    for path in filenames:
        ds = pydicom.dcmread(path)
        try:
            arr = ds.pixel_array
            if arr.shape[0] == 2 and arr.shape[1] == 1024:
                arr2 = np.zeros((3,1024,256))
                arr2[0] = arr[0]
                arr2[1] = arr[1]
                arr2[2] = arr[0] + arr[1]
                arr2 = np.transpose(arr2,[1,2,0])
                img = np.array(Image.fromarray(arr2.astype(np.uint8)).resize((64,256)))
                X.append(img)
                y.append(np.array([1,0]) if ds.PatientSex.lower() == "f" else np.array([0,1]))
            else:
                print(arr.shape)
        except:
            print(path)
    X = np.stack(X)
    y = np.stack(y)
    return X,y

x_train, y_train = get_dataset(folder + "/train")
x_test, y_test = get_dataset(folder + "/test")
print(x_train.shape, x_test.shape)
print(y_train[0:10])

(2, 256, 256)
(256, 256)
(256, 256)
C:\Users\haipw\Desktop\人工智能与医学影像\datasets/train\90243_20190820150959432000.dcm
C:\Users\haipw\Desktop\人工智能与医学影像\datasets/train\90247_20190820145151365000.dcm
(659, 256, 64, 3) (80, 256, 64, 3)
[[1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]]


In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense,Input,Flatten, GlobalMaxPooling2D
from tensorflow.keras import layers, models, Sequential, backend
base_model = tf.keras.applications.DenseNet121(
    include_top=False,
    weights="imagenet",
    input_shape=(256,64,3)
)
#  (None, 32, 8, 1024)
img_input = Input(shape=(256,64,3))
x = base_model(img_input)
x = GlobalMaxPooling2D(name='final_pooling')(x)
x = Dense(2, activation='softmax', name='fc')(x)

model = models.Model(img_input, x, name='DenseNet121')

In [8]:
model.summary()

Model: "DenseNet121"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 256, 64, 3)]      0         
                                                                 
 densenet121 (Functional)    (None, 8, 2, 1024)        7037504   
                                                                 
 final_pooling (GlobalMaxPoo  (None, 1024)             0         
 ling2D)                                                         
                                                                 
 fc (Dense)                  (None, 2)                 2050      
                                                                 
Total params: 7,039,554
Trainable params: 6,955,906
Non-trainable params: 83,648
_________________________________________________________________


Epoch 1/3
11/11 [==============================] - 127s 9s/step - loss: 1.0065 - accuracy: 0.6434 - val_loss: 1.2246 - val_accuracy: 0.6000
Epoch 2/3
11/11 [==============================] - 93s 8s/step - loss: 0.1340 - accuracy: 0.9484 - val_loss: 0.9138 - val_accuracy: 0.5750
Epoch 3/3
11/11 [==============================] - 84s 8s/step - loss: 0.0581 - accuracy: 0.9848 - val_loss: 0.8729 - val_accuracy: 0.6250


In [12]:
model.layers[1].trainable = False
for layer in model.layers:
        print(layer.name, ' is trainable? ', layer.trainable)

input_4  is trainable?  True
densenet121  is trainable?  False
final_pooling  is trainable?  True
fc  is trainable?  True


In [11]:
batch_size = 32
epochs = 10
learning_rate = 0.05
optimizer = keras.optimizers.Adadelta(learning_rate=learning_rate)
model.compile(optimizer=optimizer,
                loss='categorical_crossentropy',
                metrics=['accuracy'])
model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            verbose = 1,
            validation_data=(x_test, y_test)) 

Epoch 1/10
21/21 [==============================] - 24s 894ms/step - loss: 2.6085 - accuracy: 0.5083 - val_loss: 2.0809 - val_accuracy: 0.4750
Epoch 2/10
21/21 [==============================] - 18s 862ms/step - loss: 1.8702 - accuracy: 0.5038 - val_loss: 1.7840 - val_accuracy: 0.5375
Epoch 3/10
21/21 [==============================] - 19s 892ms/step - loss: 1.7756 - accuracy: 0.4992 - val_loss: 1.7076 - val_accuracy: 0.5125
Epoch 4/10
21/21 [==============================] - 18s 853ms/step - loss: 1.6962 - accuracy: 0.5008 - val_loss: 1.6196 - val_accuracy: 0.5000
Epoch 5/10
21/21 [==============================] - 17s 834ms/step - loss: 1.6249 - accuracy: 0.4947 - val_loss: 1.5246 - val_accuracy: 0.5500
Epoch 6/10
21/21 [==============================] - 20s 942ms/step - loss: 1.5696 - accuracy: 0.5190 - val_loss: 1.5070 - val_accuracy: 0.5250
Epoch 7/10
21/21 [==============================] - 21s 995ms/step - loss: 1.5104 - accuracy: 0.5144 - val_loss: 1.4928 - val_accuracy: 0.5125

In [13]:
loss, accuracy = model.evaluate(x_test, y_test, batch_size=128)
print("loss:", loss)
print("accuracy:", accuracy)

1/1 [==============================] - 2s 2s/step - loss: 1.3398 - accuracy: 0.5250
loss: 1.3397815227508545
accuracy: 0.5249999761581421
